# **02 Model evolution along a *P*–*T*-path**
---

This notebook shows you how to:
1. Model the system evolution of a rock along a predefined *P*–*T*-trajectory, by:
    * Automatically calling repeated Theriak minimisations.
    * Forwarding of bulk rock composition.
2. Visualisation of mineral modes an compostion as a function of *P* and *T*.
3. Fractionated phases for modelling along a *P*–*T*-path with evolving reactive bulk rock composition.

**Requirements:**
- python >= 3.10
*with the following packages installed:*
- pytheriak
- matplotlib
- ...

*Theriak-Domino back-end:*
- A working Theriak-Domino installation.
- Thermodynamic databases to use with Theriak in the working directory.
---

### Import all required libraries.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from pytheriak import wrapper

### Initiate a TherCaller-object.
The argument `programs_dir` must be set to the directory of your Theriak_Domino installation.

In [2]:
theriak = wrapper.TherCaller(programs_dir="C:\\TheriakDominoWIN\\Programs",
                             database="ds55HPT.txt",
                             theriak_version="v28.05.2022")

### Set model parameter
*P*–*T*-path from Gerya et al. (2002) in Vho et al. (2020) Fig. 2 (https://doi.org/10.5194/se-11-307-2020).\
*Bulk rock composition from Plank and Langmuir (1998) in Vho et al. (2020) (https://doi.org/10.5194/se-11-307-2020).*


In [12]:
pt_path = np.loadtxt(Path("PTpath_Gerya2002.txt")).T
# chnage to bulk from Vho et al 2020
bulk = "SI(68.2)TI(0.76)AL(25.18)FE(9.96)MN(0.02)MG(4.36)CA(0.18)NA(0.06)K(7.74)H(10)O(?)"

In [13]:
mineral_assemblage_list = []

for temperature, pressure in pt_path:
    rock, element_list = theriak.minimisation(pressure, temperature, bulk, return_failed_minimisation=True)
    mineral_assemblage_list.append(rock.mineral_assemblage)

In [21]:
mineral_names = [[mineral.name for mineral in mineral_assemblage] for mineral_assemblage  in mineral_assemblage_list]
mineral_names[0:3]

[['WM02_mu', 'GT07W2_alm', 'BIOTI_obi', 'ky', 'mic', 'q', 'ru'],
 ['WM02_mu', 'GT07W2_alm', 'BIOTI_obi', 'ky', 'mic', 'q', 'ru'],
 ['WM02_mu', 'GT07W2_alm', 'BIOTI_obi', 'ky', 'mic', 'q', 'ru']]

In [23]:
mineral_modes = [[mineral.vol_percent for mineral in mineral_assemblage] for mineral_assemblage  in mineral_assemblage_list]
mineral_modes[0:3]

[[22.6212, 16.2924, 10.5029, 4.8195, 12.4651, 32.8321, 0.4667],
 [24.6232, 17.6778, 8.2195, 4.1358, 12.6259, 32.2242, 0.4937],
 [26.9243, 19.2789, 5.518, 3.3865, 12.876, 31.4803, 0.536]]

In [20]:
mineral_em_act = [[mineral.endmember_activities for mineral in mineral_assemblage if mineral.solution_phase] for mineral_assemblage  in mineral_assemblage_list]
mineral_em_act[0]

[{'mu': 0.796626, 'pa': 0.313446, 'cel': 0.0661045, 'fcel': 0.0187337},
 {'alm': 0.654021, 'py': 0.00217107, 'gr': 0.000217685, 'spss': 8.40682e-09},
 {'phl': 0.198932,
  'ann': 0.0266945,
  'obi': 0.378393,
  'east': 0.0147268,
  'tbi': 0.000641722,
  'mnbi': 2.54908e-16}]

In [ ]:
fig, ax = plt.subplots()

